## Object Tracking using optical flow

### Lucas-Kanade
 * to track sparse point

 * using corner detection

In [1]:
import numpy as np
import cv2

In [2]:
corner_track_params = dict(maxCorners = 10,qualityLevel=0.3,minDistance=7,blockSize=7)

In [3]:
lk_params = dict(winSize=(200,200),maxLevel=2,criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))
#winSize if small = more sensitive to noise and miss larger motions (that means miss points if objext is moving fast)
#and vise-versa for larger window size

In [6]:
cap = cv2.VideoCapture('VID_20200207_150422.mp4')

ret,pre_frame = cap.read()
pre_gray = cv2.cvtColor(pre_frame,cv2.COLOR_BGR2GRAY)

#pts to track
prevPts = cv2.goodFeaturesToTrack(pre_gray,mask=None,**corner_track_params)

mask = np.zeros_like(pre_frame)
while True:
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #calculate optical flow on next frame
    nextPts,status,err = cv2.calcOpticalFlowPyrLK(pre_gray,frame_gray,prevPts,None,**lk_params)
    goog_new = nextPts[status==1]
    good_prev = nextPts[status==1]
    
    for i,(new,prev) in enumerate(zip(goog_new,good_prev)):
        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()
        
        mask = cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)
        
        frame = cv2.circle(frame,(x_new,y_new),8,(255,0,0),-1)
        
    img = cv2.add(frame,mask)
    cv2.imshow('tracking',img)
    
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break
        
    #update previous points to be the new points
    #update the new frame to be previous frame
    pre_gray = frame_gray.copy()
    prevPts = goog_new.reshape(-1,1,2)
    
cv2.destroyAllWindows()
cap.release()